In [ ]:
!pip install --q "snowflake-connector-python[pandas]"
!sudo pip install --q  snowflake-ml-python==1.0.11 -U
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip install --q seaborn
!pip install --upgrade --q xgboost==1.7.3
!pip install --upgrade --q numpy==1.24.3
!pip install --upgrade --q pandas==1.5.3
!pip install --upgrade --q anyio==3.5.0
!pip install --upgrade --q packaging==23.1
!pip install --upgrade --q scikit-learn==1.3.0
!pip install --upgrade --q typing-extensions==4.7.1
!pip install --upgrade --q cryptography==39.0.0
!pip install --upgrade --q fsspec==2023.9.2
!pip install --q xgboost
!pip install --q catboost
!pip install --q refractml

^C
ERROR: Operation cancelled by user
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
import configparser

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier


from xgboost import XGBClassifier
from catboost import CatBoostClassifier

Matplotlib created a temporary cache directory at /tmp/matplotlib-ej_1pi4c because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [6]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [7]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [8]:
df = session.table("AUTO_INSURANCE_CLAIMS_DATA_PRODUCT")
insurance_claim = df.to_pandas()

In [9]:
insurance_claim.columns = insurance_claim.columns.str.lower()

In [10]:
insurance_claim.columns

Index(['months_as_customer', 'customer_age', 'policy_number',
       'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day', 'incident_time_of_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount_paid',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported'],
      dtype='object')

In [11]:
insurance_claim.rename(columns = {'total_claim_amount_paid': 'total_claim_amount'}, inplace=True)

In [12]:
insurance_claim=insurance_claim.replace("?",np.NaN)

In [13]:
df = insurance_claim.copy()

In [14]:
# dropping columns which are not necessary for prediction

to_drop = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year']

df.drop(to_drop, inplace=True, axis=1)

In [15]:
# separating the feature and target columns

X = df.drop('fraud_reported', axis = 1)
y = df['fraud_reported']

In [16]:
# extracting categorical columns
cat_df = X.select_dtypes(include = ['object'])

In [17]:
cat_df.columns

Index(['customer_age', 'policy_csl', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_relationship', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'incident_time_of_day', 'property_damage', 'police_report_available'],
      dtype='object')

In [18]:
# printing unique values of each column
for col in cat_df.columns:
    print(f"{col}: \n{cat_df[col].unique()}\n")

customer_age: 
['58' '31' '54' '61' '41' '32' '48' '24' '66' '57' '62' '56' '68' '26'
 '28' '64' '43' '46' '42' '21' '49' '59' '52' '47' '22' '63' '65' '69'
 '35' '37' '60' '30' '40' '20' '39' '45' '19' '67' '33' '29' '44' '34'
 '23' '53' '38' '27' '50' '55' '36' '25' '51']

policy_csl: 
['250/500' '100/300' '500/1000']

insured_sex: 
['FEMALE' 'MALE']

insured_education_level: 
['MD' 'College' 'Masters' 'Associate' 'High School' 'PhD' 'JD']

insured_occupation: 
['tech-support' 'adm-clerical' 'sales' 'machine-op-inspct'
 'prof-specialty' 'farming-fishing' 'exec-managerial' 'priv-house-serv'
 'handlers-cleaners' 'other-service' 'armed-forces' 'craft-repair'
 'transport-moving' 'protective-serv']

insured_relationship: 
['other-relative' 'husband' 'wife' 'own-child' 'not-in-family' 'unmarried']

incident_type: 
['Multi-vehicle Collision' 'Single Vehicle Collision' 'Parked Car'
 'Vehicle Theft']

collision_type: 
['Details not Available' 'Front Collision' 'Rear Collision'
 'Side Collisio

In [19]:
cat_df = pd.get_dummies(cat_df, drop_first = True)

In [20]:
num_df = df.select_dtypes(include = ['int16','int8','int32','float64'])

In [21]:
num_df.columns

Index(['months_as_customer', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'capital_gains', 'capital_loss',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim',
       'property_claim', 'vehicle_claim'],
      dtype='object')

In [22]:
# combining the Numerical and Categorical dataframes to get the final dataset

X = pd.concat([num_df, cat_df], axis = 1)

In [23]:
# splitting data into training set and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [24]:
num_df = X_train[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [25]:
num_df_test = X_test[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [26]:
# Scaling the numeric values in the dataset

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(num_df)

In [27]:
scaled_data_test = scaler.transform(num_df_test)

In [28]:
scaled_num_df = pd.DataFrame(data = scaled_data, columns = num_df.columns, index = X_train.index)
scaled_num_df.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
78260,-0.922218,-1.286837,1.014694,-0.712308,-0.455184,-0.008822,-0.678643,1.227446,1.343199,0.050396,1.700563,0.835867
193891,1.980096,0.252036,1.369086,1.131135,0.123003,0.503033,0.747867,0.004975,1.343199,0.066058,0.403058,-0.466528
114736,-0.911224,-1.260659,0.710815,-0.872429,-0.554963,1.014889,2.174376,0.004975,0.446423,1.957127,-0.984640,1.321046
225365,-0.779301,-0.217292,-0.395344,0.072997,-0.797714,-0.008822,-0.678643,-1.217496,-0.450353,-1.009878,-0.940178,0.112690
122050,0.056214,0.758761,0.632595,-0.155629,0.827875,0.332415,-0.678643,1.227446,1.343199,1.981180,-0.948562,-0.543544


In [29]:
scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X_test.index)
scaled_num_df_test.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
125868,1.067626,1.200041,-0.113806,1.578596,0.193146,1.185507,-0.678643,0.004975,-1.347129,1.486423,-0.970920,-1.329138
121180,1.782211,0.482025,-0.386623,0.445437,1.383184,-0.179441,0.747867,0.004975,-1.347129,1.209258,-0.977526,-1.454928
54606,-0.856256,0.798027,-0.623614,-0.966745,1.199452,-0.691297,2.174376,-1.217496,-1.347129,-1.023862,-0.958725,1.539727
120570,-0.834269,1.839524,-0.994375,-0.414241,0.205781,1.356126,-0.678643,1.227446,-0.450353,-0.865702,-0.979812,-1.289546
50824,1.034646,1.987241,1.391252,-1.715945,-0.333242,1.014889,2.174376,0.004975,0.446423,0.601649,-0.946149,-0.806153


In [30]:
X_train.drop(columns = scaled_num_df.columns, inplace = True)

In [31]:
X_train = pd.concat([scaled_num_df, X_train], axis = 1)

In [32]:
X_test.drop(columns = scaled_num_df_test.columns, inplace = True)

In [33]:
X_test = pd.concat([scaled_num_df_test, X_test], axis = 1)

# Stochastic Gradient Boosting

In [55]:
sgb = GradientBoostingClassifier(subsample = 0.90, max_features = 0.70)
sgb.fit(X_train, y_train)

GradientBoostingClassifier(max_features=0.7, subsample=0.9)

In [51]:
y_pred = sgb.predict(X_test)

In [52]:
y_prob = sgb.predict_proba(X_test)[:,1]

In [53]:
sgb_train_acc = accuracy_score(y_train, sgb.predict(X_train))
sgb_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of CatBoost is : {sgb_train_acc}")
print(f"Test accuracy of CatBoost is : {sgb_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of CatBoost is : 0.8528554679051554
Test accuracy of CatBoost is : 0.8529376611402325
[[   14  8613]
 [    0 49940]]
                   precision    recall  f1-score   support

   Fraud Reported       1.00      0.00      0.00      8627
No Fraud Reported       0.85      1.00      0.92     49940

         accuracy                           0.85     58567
        macro avg       0.93      0.50      0.46     58567
     weighted avg       0.87      0.85      0.79     58567



# CATBoost

In [56]:
cat = CatBoostClassifier(iterations=10)
cat.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.4773611	total: 90.8ms	remaining: 817ms
1:	learn: 0.4262490	total: 179ms	remaining: 717ms
2:	learn: 0.4163634	total: 264ms	remaining: 616ms
3:	learn: 0.4134074	total: 292ms	remaining: 438ms
4:	learn: 0.4113457	total: 368ms	remaining: 368ms
5:	learn: 0.4106672	total: 394ms	remaining: 263ms
6:	learn: 0.4093667	total: 473ms	remaining: 203ms
7:	learn: 0.4086592	total: 555ms	remaining: 139ms
8:	learn: 0.4080474	total: 586ms	remaining: 65.2ms
9:	learn: 0.4075486	total: 670ms	remaining: 0us


In [57]:
cat_acc = accuracy_score(y_test, cat.predict(X_test))

In [58]:
print(f"Training Accuracy of Cat Boost Classifier is {accuracy_score(y_train, cat.predict(X_train))}")
print(f"Test Accuracy of Cat Boost Classifier is {cat_acc} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, cat.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, cat.predict(X_test))}")

Training Accuracy of Cat Boost Classifier is 0.8525424307618755
Test Accuracy of Cat Boost Classifier is 0.8528522888315946 

Confusion Matrix :- 
[[    9  8618]
 [    0 49940]]

Classification Report :- 
                    precision    recall  f1-score   support

   Fraud Reported       1.00      0.00      0.00      8627
No Fraud Reported       0.85      1.00      0.92     49940

         accuracy                           0.85     58567
        macro avg       0.93      0.50      0.46     58567
     weighted avg       0.87      0.85      0.79     58567



# Gradient Boosting

In [59]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

GradientBoostingClassifier()

In [60]:
gb_acc = accuracy_score(y_test, gb.predict(X_test))

In [61]:
print(f"Training Accuracy of Gradient Boosting Classifier is {accuracy_score(y_train, gb.predict(X_train))}")
print(f"Test Accuracy of Gradient Boosting Classifier is {gb_acc} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, gb.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, gb.predict(X_test))}")

Training Accuracy of Gradient Boosting Classifier is 0.8523261505537911
Test Accuracy of Gradient Boosting Classifier is 0.8527669165229567 

Confusion Matrix :- 
[[    4  8623]
 [    0 49940]]

Classification Report :- 
                    precision    recall  f1-score   support

   Fraud Reported       1.00      0.00      0.00      8627
No Fraud Reported       0.85      1.00      0.92     49940

         accuracy                           0.85     58567
        macro avg       0.93      0.50      0.46     58567
     weighted avg       0.87      0.85      0.79     58567



# Adaboost

In [ ]:
ada = AdaBoostClassifier(n_estimators=100, random_state=0)
ada.fit(X_train, y_train)

In [ ]:
ada_acc = accuracy_score(y_test, ada.predict(X_test))

In [ ]:
print(f"Training Accuracy of Adaboosting Classifier is {accuracy_score(y_train, ada.predict(X_train))}")
print(f"Test Accuracy of Adaboosting Classifier is {ad_acc} \n")

print(f"Confusion Matrix :- \n{confusion_matrix(y_test, ada.predict(X_test))}\n")
print(f"Classification Report :- \n {classification_report(y_test, ada.predict(X_test))}")

In [35]:
from refractml import *
from refractml.constants import MLModelFlavours
import requests

In [36]:
actual_model_columns = ['months_as_customer',
 'policy_deductable',
 'umbrella_limit',
 'capital_gains',
 'capital_loss',
 'incident_hour_of_the_day',
 'number_of_vehicles_involved',
 'bodily_injuries',
 'witnesses',
 'injury_claim',
 'property_claim',
 'vehicle_claim',
 'policy_annual_premium',
 'total_claim_amount',
 'customer_age_20',
 'customer_age_21',
 'customer_age_22',
 'customer_age_23',
 'customer_age_24',
 'customer_age_25',
 'customer_age_26',
 'customer_age_27',
 'customer_age_28',
 'customer_age_29',
 'customer_age_30',
 'customer_age_31',
 'customer_age_32',
 'customer_age_33',
 'customer_age_34',
 'customer_age_35',
 'customer_age_36',
 'customer_age_37',
 'customer_age_38',
 'customer_age_39',
 'customer_age_40',
 'customer_age_41',
 'customer_age_42',
 'customer_age_43',
 'customer_age_44',
 'customer_age_45',
 'customer_age_46',
 'customer_age_47',
 'customer_age_48',
 'customer_age_49',
 'customer_age_50',
 'customer_age_51',
 'customer_age_52',
 'customer_age_53',
 'customer_age_54',
 'customer_age_55',
 'customer_age_56',
 'customer_age_57',
 'customer_age_58',
 'customer_age_59',
 'customer_age_60',
 'customer_age_61',
 'customer_age_62',
 'customer_age_63',
 'customer_age_64',
 'customer_age_65',
 'customer_age_66',
 'customer_age_67',
 'customer_age_68',
 'customer_age_69',
 'policy_csl_250/500',
 'policy_csl_500/1000',
 'insured_sex_MALE',
 'insured_education_level_College',
 'insured_education_level_High School',
 'insured_education_level_JD',
 'insured_education_level_MD',
 'insured_education_level_Masters',
 'insured_education_level_PhD',
 'insured_occupation_armed-forces',
 'insured_occupation_craft-repair',
 'insured_occupation_exec-managerial',
 'insured_occupation_farming-fishing',
 'insured_occupation_handlers-cleaners',
 'insured_occupation_machine-op-inspct',
 'insured_occupation_other-service',
 'insured_occupation_priv-house-serv',
 'insured_occupation_prof-specialty',
 'insured_occupation_protective-serv',
 'insured_occupation_sales',
 'insured_occupation_tech-support',
 'insured_occupation_transport-moving',
 'insured_relationship_not-in-family',
 'insured_relationship_other-relative',
 'insured_relationship_own-child',
 'insured_relationship_unmarried',
 'insured_relationship_wife',
 'incident_type_Parked Car',
 'incident_type_Single Vehicle Collision',
 'incident_type_Vehicle Theft',
 'collision_type_Front Collision',
 'collision_type_Rear Collision',
 'collision_type_Side Collision',
 'incident_severity_Minor Damage',
 'incident_severity_Total Loss',
 'authorities_contacted_Fire',
 'authorities_contacted_None',
 'authorities_contacted_Other',
 'authorities_contacted_Police',
 'incident_time_of_day_Early Morning Hours',
 'incident_time_of_day_Morning to Noon',
 'incident_time_of_day_Night Time',
 'property_damage_No Property Damage',
 'property_damage_Property Damage',
 'police_report_available_Police Report Available',
 'police_report_available_Unknown']

In [37]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    df = pd.DataFrame(payload_dict,index=[0])
    df = df.fillna(0)
    df.columns = df.columns.str.lower()
    df.drop(to_drop, inplace=True, axis=1)
    
    # extracting categorical columns
    cat_df = df[['customer_age','policy_csl','insured_sex','insured_education_level','insured_occupation','insured_relationship','incident_type','collision_type','incident_severity','authorities_contacted','incident_time_of_day','property_damage','police_report_available']]
    cat_df = pd.get_dummies(cat_df, drop_first = True)

    num_df = df[['months_as_customer', 'policy_deductable', 'policy_annual_premium',
       'umbrella_limit', 'capital_gains', 'capital_loss',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim',
       'property_claim', 'vehicle_claim']]
    
    X = pd.concat([num_df, cat_df], axis = 1)
    
    num_df_test = X[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

    scaled_data_test = scaler.transform(num_df_test)
    scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X.index)
    
    X.drop(columns = scaled_num_df_test.columns, inplace = True)
    X = pd.concat([scaled_num_df_test, X], axis = 1)
    
    missing_features = [missing_col for missing_col in actual_model_columns if missing_col not in X.columns]
    X[missing_features] = 0
    
    prediction = model.predict(X[actual_model_columns])
    probability = model.predict_proba(X)[:,1]
    return {"prediction" : prediction, "probability" : probability}

In [38]:
insurance_claim.columns = insurance_claim.columns.str.upper()
insurance_claims = insurance_claim.copy()
insurance_claims.drop('FRAUD_REPORTED', axis = 1, inplace=True)
payload = insurance_claims.iloc[0].to_dict()

In [40]:
req = requests.Request()
req.json = {"payload":payload}
y_req = req
y_out = score(etc, y_req)
y_out

{'prediction': array(['No Fraud Reported'], dtype=object),
 'probability': array([1.])}

In [41]:
req.json

{'payload': {'MONTHS_AS_CUSTOMER': 37,
  'CUSTOMER_AGE': '58',
  'POLICY_NUMBER': 459791,
  'POLICY_BIND_DATE': '27-04-2010',
  'POLICY_STATE': 'CT',
  'POLICY_CSL': '250/500',
  'POLICY_DEDUCTABLE': 1171,
  'POLICY_ANNUAL_PREMIUM': 1394,
  'UMBRELLA_LIMIT': 4344805,
  'INSURED_ZIP': '444558',
  'INSURED_SEX': 'FEMALE',
  'INSURED_EDUCATION_LEVEL': 'MD',
  'INSURED_OCCUPATION': 'tech-support',
  'INSURED_HOBBIES': 'board-games',
  'INSURED_RELATIONSHIP': 'other-relative',
  'CAPITAL_GAINS': 77256,
  'CAPITAL_LOSS': -9989,
  'INCIDENT_DATE': datetime.date(2022, 5, 13),
  'INCIDENT_TYPE': 'Multi-vehicle Collision',
  'COLLISION_TYPE': 'Details not Available',
  'INCIDENT_SEVERITY': 'Major Damage',
  'AUTHORITIES_CONTACTED': 'None',
  'INCIDENT_STATE': 'CT',
  'INCIDENT_CITY': 'Vernon',
  'INCIDENT_LOCATION': 'Lane',
  'INCIDENT_HOUR_OF_THE_DAY': 22,
  'INCIDENT_TIME_OF_DAY': 'Night Time',
  'NUMBER_OF_VEHICLES_INVOLVED': 3,
  'PROPERTY_DAMAGE': 'No Property Damage',
  'BODILY_INJURIES': 

In [42]:
## registering the model in refract.
model_reg = register_model(etc, 
               score, 
               name="ILF_ExtraTree_FraudClassifier", 
               description="Insurance claim's model trained for Fraud Classification",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=y_pred, 
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
